# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a websitt1. It allows you to understand how users interact with the sitt1. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_page (string)` - Page on the sitt1.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the sitt1.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leveragt1. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.t1.: no O^2 or O^3 complexities; appropriate object usage; no data leaks ett0. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code ett0. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql import Row
import findspark
findspark.init()
from pyspark.sql.functions import col, concat_ws, collect_list, expr, first, lit, min as spark_min
from pyspark.sql.types import ArrayType, StringType
import pandas as pd

In [2]:
spark = SparkSession.builder.appName("Maykov").getOrCreate()

/opt/conda/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [3]:
! hdfs dfs -head /data/clickstream.csv

user_id	session_id	event_type	event_page	timestamp
562	507	page	main	1695584127
562	507	event	main	1695584134
562	507	event	main	1695584144
562	507	event	main	1695584147
562	507	wNaxLlerrorU	main	1695584154
562	507	event	main	1695584154
562	507	event	main	1695584154
562	507	event	main	1695584160
562	507	page	rabota	1695584166
562	507	event	rabota	1695584174
562	507	event	rabota	1695584181
562	507	event	rabota	1695584189
562	507	page	main	1695584194
562	507	event	main	1695584204
562	507	event	main	1695584211
562	507	event	main	1695584211
562	507	event	main	1695584219
562	507	page	bonus	1695584221
562	507	page	online	1695584222
562	507	event	online	1695584230
3539	849	page	main	1695584238
3539	849	event	main	1695584252
3539	849	page	online	1695584261
3539	849	page	bonus	1695584269
3539	849	event	bonus	1695584278
3539	849	page	news	1695584285
3539	849	page	main	1695584291
3539	849	event	main	1695584301
3539	849	page	news	1695584306
3539	849	event	news	1695584307
3539	849	page	vklad	169558

<h2>Spark SQL</h2>

In [4]:
sql_tab = f"""CREATE OR REPLACE TEMPORARY VIEW clnt_df USING csv OPTIONS ('path' '{"hdfs:/data/clickstream.csv"}','header' 'true','sep' '\t')"""

In [5]:
sql_1 = """
SELECT distinct event_type
FROM clnt_df
where upper(event_type) not like '%ERROR%'
limit 30;
"""

In [6]:
spark.sql(sql_tab)
sql_result = spark.sql(sql_1)
sql_result.show(30, truncate=False)

+----------+
|event_type|
+----------+
|page      |
|event     |
+----------+



except for the '%error%' like event types there are only 'page' and 'event' phases, so either "in ('page', 'event')" or "not like '%error%'" could be used, I will prefer the second option, as if the base will be ipdated with some new values the logic will not fail to catch every non error event. Moreover, I will use upper function to make the text of the same format and catch an '%ERROR%' even if it has a strange forms like '%eRror%'.

In [7]:
sql_main_1 = """
with fin_table_0 as (
select t0.user_id, t0.session_id, min(t0.timestamp) as min_error_time
from clnt_df t0
where upper(t0.event_type) like '%ERROR%'
group by 1,2
)
, fin_table_1 as ( 
select t0.user_id, t0.session_id, t0.event_page, t0.timestamp
from clnt_df t0
left join fin_table_0 t1
on t0.user_id = t1.user_id and t0.session_id = t1.session_id
where upper(t0.event_type) not like '%ERROR%'
and (t1.min_error_time is null or t0.timestamp < t1.min_error_time)
)
, fin_table_2 as (
select distinct t0.user_id, t0.session_id, collect_list(t0.event_page) over (partition by t0.user_id, t0.session_id order by t0.timestamp, t0.event_page rows between unbounded preceding and unbounded following) as event_pages
from fin_table_1 t0
ORDER BY 1,2
)
, fin_table_3 as (
select t0.user_id, t0.session_id, t0.event_pages,
transform(t0.event_pages, (page, i) -> 
if(i = 0 or page != t0.event_pages[i - 1], page, null)
) as unique_event_pages
from fin_table_2 t0
)
, fin_table as (
select distinct t0.user_id, t0.session_id, concat_ws('-', t0.unique_event_pages) as route
from fin_table_3 t0
)

select route, count(session_id) as count
from fin_table
group by 1
order by 2 desc
limit 30;
"""

In [8]:
spark.sql(sql_tab)
spark_sql_result = spark.sql(sql_main_1)
spark_sql_result.show(30, truncate=False)

+---------------------+-----+
|route                |count|
+---------------------+-----+
|main                 |8184 |
|main-archive         |1112 |
|main-rabota          |1047 |
|main-internet        |895  |
|main-bonus           |869  |
|main-news            |769  |
|main-tariffs         |677  |
|main-online          |587  |
|main-vklad           |518  |
|main-rabota-archive  |170  |
|main-archive-rabota  |167  |
|main-bonus-archive   |143  |
|main-rabota-bonus    |138  |
|main-bonus-rabota    |135  |
|main-news-rabota     |135  |
|main-archive-internet|132  |
|main-rabota-news     |130  |
|main-internet-rabota |129  |
|main-archive-news    |126  |
|main-rabota-internet |124  |
|main-internet-archive|123  |
|main-archive-bonus   |117  |
|main-internet-bonus  |115  |
|main-tariffs-internet|114  |
|main-news-archive    |113  |
|main-news-internet   |108  |
|main-archive-tariffs |104  |
|main-internet-news   |103  |
|main-tariffs-archive |103  |
|main-rabota-main     |94   |
+---------

Export the results:

In [9]:
spark_sql_result.toPandas().to_csv("spark_sql_result.csv", index=False, sep='\t')

<h2>Spark RDD</h2>

As was previously mentioned except for the '%error%' like event types there are only 'page' and 'event' phases, so I have filtered such events ".upper() in ('PAGE', 'EVENT')". Moreover, I will use upper function to make the text of the same format and catch an '%ERROR%' even if it has a strange forms like '%eRror%' like it was with spark SQL.

In [10]:
rdd_tab = spark.sparkContext.textFile("hdfs:/data/clickstream.csv").map(lambda i: i.split('\t')).persist()

fin_tab_0 = rdd_tab.filter(lambda i: 'ERROR' in i[2].upper()).map(lambda i: ((i[0], i[1]), i[4])).reduceByKey(lambda x, y: min(x, y))
fin_tab_1 = rdd_tab.filter(lambda i: i[2].upper() in ('PAGE', 'EVENT')).map(lambda i: ((i[0], i[1]), (i[3], i[4]))).leftOuterJoin(fin_tab_0).filter(lambda i: i[1][1] is None or (i[1][0][1] < i[1][1]))
fin_tab_2 = fin_tab_1.map(lambda i: ((i[0][0], i[0][1]), (i[1][0][0], i[1][0][1]))).groupByKey().mapValues(lambda j: [page for page, _ in sorted(j, key=lambda y: (y[1], y[0]))])

def remove_subsequent_repetitions(route):
    return '-'.join([route[i] for i in range(len(route)) if i == 0 or route[i] != route[i - 1]])

unique_routes = fin_tab_2.mapValues(remove_subsequent_repetitions)
route_counts = unique_routes.map(lambda i: (i[1], 1)).reduceByKey(lambda x, y: x + y)
spark_rdd_result = spark.sparkContext.parallelize(route_counts.takeOrdered(30, key=lambda x: -x[1])).map(lambda i: (i[0], i[1]))
spark_rdd_result.collect()

[('main', 8184),
 ('main-archive', 1112),
 ('main-rabota', 1047),
 ('main-internet', 895),
 ('main-bonus', 869),
 ('main-news', 769),
 ('main-tariffs', 677),
 ('main-online', 587),
 ('main-vklad', 518),
 ('main-rabota-archive', 170),
 ('main-archive-rabota', 167),
 ('main-bonus-archive', 143),
 ('main-rabota-bonus', 138),
 ('main-news-rabota', 135),
 ('main-bonus-rabota', 135),
 ('main-archive-internet', 132),
 ('main-rabota-news', 130),
 ('main-internet-rabota', 129),
 ('main-archive-news', 126),
 ('main-rabota-internet', 124),
 ('main-internet-archive', 123),
 ('main-archive-bonus', 117),
 ('main-internet-bonus', 115),
 ('main-tariffs-internet', 114),
 ('main-news-archive', 113),
 ('main-news-internet', 108),
 ('main-archive-tariffs', 104),
 ('main-internet-news', 103),
 ('main-tariffs-archive', 103),
 ('main-rabota-main', 94)]

Export the results:

In [11]:
spark_rdd_result = spark_rdd_result.map(lambda i: Row(route=i[0], count=i[1])).toDF()
spark_rdd_result.write.mode("overwrite").csv("spark_rdd_result.csv", header=True)

<h2>Spark DF</h2>

As was previously mentioned except for the '%error%' like event types there are only 'page' and 'event' phases, so I have filtered such events ".isin("page", "event")".

In [12]:
df = spark.read.option("delimiter", "\t").csv("hdfs:/data/clickstream.csv", header=True)
fin_tab_0 = (df.filter(F.col("event_type").contains("error")).groupBy("user_id", "session_id").agg(F.min("timestamp").alias("min_error_time")))
fin_tab_1 = (df.alias("t0").join(fin_tab_0.alias("t1"), (F.col("t0.user_id") == F.col("t1.user_id")) & (F.col("t0.session_id") == F.col("t1.session_id")), "left_outer")
.filter((F.col("t0.event_type").isin("page", "event")) & ((F.col("t1.min_error_time").isNull()) | (F.col("t0.timestamp") < F.col("t1.min_error_time"))))
.select("t0.user_id", "t0.session_id", "t0.event_page", "t0.timestamp"))

def remove_consecutive_repetitions(event_pages):
    return [event_pages[i] for i in range(len(event_pages)) if i == 0 or event_pages[i] != event_pages[i - 1]]

remove_repetitions_udf = F.udf(remove_consecutive_repetitions, ArrayType(StringType()))
fin_tab_2 = (fin_tab_1.withColumn("event_pages", remove_repetitions_udf(F.collect_list("event_page").over(Window.partitionBy("user_id", "session_id")
.orderBy("timestamp", "event_page").rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing))))
.select("user_id", "session_id", F.concat_ws('-', "event_pages").alias("route")).distinct())

spark_df_result = (fin_tab_2.groupBy("route").agg(F.count("session_id").alias("count")).orderBy(F.desc("count")).limit(30))
spark_df_result.show(30, truncate=False)

+---------------------+-----+
|route                |count|
+---------------------+-----+
|main                 |8184 |
|main-archive         |1112 |
|main-rabota          |1047 |
|main-internet        |895  |
|main-bonus           |869  |
|main-news            |769  |
|main-tariffs         |677  |
|main-online          |587  |
|main-vklad           |518  |
|main-rabota-archive  |170  |
|main-archive-rabota  |167  |
|main-bonus-archive   |143  |
|main-rabota-bonus    |138  |
|main-bonus-rabota    |135  |
|main-news-rabota     |135  |
|main-archive-internet|132  |
|main-rabota-news     |130  |
|main-internet-rabota |129  |
|main-archive-news    |126  |
|main-rabota-internet |124  |
|main-internet-archive|123  |
|main-archive-bonus   |117  |
|main-internet-bonus  |115  |
|main-tariffs-internet|114  |
|main-news-archive    |113  |
|main-news-internet   |108  |
|main-archive-tariffs |104  |
|main-internet-news   |103  |
|main-tariffs-archive |103  |
|main-rabota-main     |94   |
+---------

Export the results:

In [13]:
spark_df_result.toPandas().to_csv("spark_df_result.csv", index=False, sep='\t')

<b>Comparison between the results obtained:</b>

In [14]:
compare_all = pd.concat([df.toPandas() for df in [spark_sql_result, spark_rdd_result, spark_df_result]], axis=1).set_index('route')
compare_all['compare'] = (compare_all.iloc[:, -3:].nunique(axis=1) == 1)
compare_all

,count,count,count,compare
route,,,,
"(main, main, main)",8184,8184,8184,True
"(main-archive, main-archive, main-archive)",1112,1112,1112,True
"(main-rabota, main-rabota, main-rabota)",1047,1047,1047,True
"(main-internet, main-internet, main-internet)",895,895,895,True
"(main-bonus, main-bonus, main-bonus)",869,869,869,True
"(main-news, main-news, main-news)",769,769,769,True
"(main-tariffs, main-tariffs, main-tariffs)",677,677,677,True
"(main-online, main-online, main-online)",587,587,587,True
"(main-vklad, main-vklad, main-vklad)",518,518,518,True


As could be seen the resulted counts are the same for all 3 methods and the logic is consistent between the spark SQL, RDD, and DF.

In [15]:
spark.stop()